In [1]:
import json
import pymongo
from bson.objectid import ObjectId
from tqdm import tqdm 

In [2]:
# Cargamos el fichero json de documentos
with open('./dblp.json') as json_file:
    json_documents = json.load(json_file)

In [3]:
# Comprobación del número de documentos de la base de datos
len(json_documents)

4868818

In [5]:
# Conexión al servidor local de mongodb
conex = pymongo.MongoClient()
conex.list_database_names()

['BDClientes',
 'BDPacientes',
 'BDSuCasa',
 'Provincias',
 'admin',
 'clasesabadoBD',
 'config',
 'local',
 'practica',
 'prueba']

In [8]:
db = conex.practica

In [9]:
# Conexión a la base de datos e inserción en la colección de documentos
for k, v in tqdm(json_documents.items()):
    db.documentos.insert_one(v)

100%|██████████| 4868818/4868818 [25:22<00:00, 3197.21it/s]


In [10]:
db.list_collection_names()

['documentos']

In [11]:
# Ejemplo de documento de la colección documentos
db.documentos.find_one()

{'_id': 'tr/meltdown/s18',
 'authors': ['Paul Kocher',
  'Daniel Genkin',
  'Daniel Gruss',
  'Werner Haas',
  'Mike Hamburg',
  'Moritz Lipp',
  'Stefan Mangard',
  'Thomas Prescher 0002',
  'Michael Schwarz 0001',
  'Yuval Yarom'],
 'title': 'Spectre Attacks: Exploiting Speculative Execution.',
 'year': '2018',
 'type': 'article'}

In [12]:
# Creación de la colección de autores
pipeline_authors_collection = [{"$unwind": "$authors"}, {"$group": {"_id": "$authors", "publications": {"$push": "$_id"}}}, {'$out': "autores"}]
db.documentos.aggregate(pipeline_authors_collection,  allowDiskUse = True)

In [9]:
# Se muestran 5 documentos de la nueva colección autores
pipeline = [{"$sample": {"size": 5}}]
cursor = db.autores.aggregate(pipeline)
for doc in list(cursor):
    print(doc)

{'_id': 'Elke Zimmermann', 'publications': ['journals/bmcmi/EndersZRMKAKDBTHD11']}
{'_id': 'Edwin Walter Reid', 'publications': ['journals/tim/ReidOGM12']}
{'_id': 'Bianca Dumitrascu', 'publications': ['journals/bioinformatics/DumitrascuDAE19', 'journals/corr/abs-1905-10003', 'journals/corr/abs-1805-07458', 'journals/corr/abs-1906-00226']}
{'_id': 'Mohammad Jamil Mismar', 'publications': ['journals/ijes/ElissaM19', 'journals/wpc/Abu-Al-NadiIAM12']}
{'_id': 'Michael Painter', 'publications': ['journals/corr/abs-2003-04445']}


In [12]:
db.list_collection_names()

['autores', 'documentos']

In [7]:
# En caso de querer borrar las colleciones y/o la base de datos
db.documentos.drop()
db.autores.drop()
conex.drop_database('practica')